<a href="https://colab.research.google.com/github/TUNIX17/LISTA/blob/main/Hospital_stay_in_days.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: upload file

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  # Do something with the uploaded file, e.g.:
  #   with open(fn, 'wb') as f:
  #     f.write(uploaded[fn])


In [ ]:
# prompt: show schema

import pandas as pd

# Read the uploaded CSV file into a DataFrame
df = pd.read_csv(fn)

# Print the schema of the DataFrame
print(df.info())


In [ ]:
# Import necessary libraries
import pandas as pd
import lightgbm as lgb
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score

# Load the dataset
df = pd.read_csv('hospital.csv')

# Define features and target
X = df.drop(columns=['Stay (in days)'])
y = df['Stay (in days)']

# Define categorical features
categorical_features = ['Department', 'Ward_Facility_Code', 'doctor_name', 'gender', 'Type of Admission', 'Severity of Illness', 'Age']

# Preprocessing for categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Define the LightGBM model with the given hyperparameters
model = lgb.LGBMRegressor(
    boosting_type='gbdt',
    num_leaves=50,
    learning_rate=0.1,
    n_estimators=101,
    min_child_samples=30,
    subsample=1,
    colsample_bytree=0.9
)

# Create a pipeline that includes the preprocessor and the model
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred = pipeline.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R-squared score: {r2}')

# Save the model
joblib.dump(pipeline, 'model.pkl')
